1. Toma tu tabla `FactVentas` de PostgreSQL.
2. Expórtala a un CSV con Python.
3. Simula que la subes a la nube:
   * Crea un nuevo esquema dw_cloud en PostgreSQL y carga ahí el CSV allí, como si fuera tu "Data Warehouse en la nube".
4. Haz una consulta SQL de ejemplo sobre esa tabla para obtener:
   * Top 3 clientes con más ventas.
   * Ventas totales por mes.

### Take `FactVentas` table from PostgreSQL and export in CSV file with Python

In [1]:
# Libraries
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

# conection config
config = {
    'host': 'localhost',
    'port':'5432',
    'database':'data_warehouse',
    'user': 'postgres',
    'password': 'root'
}


try:
    # Create connection
    connection = f"postgresql://{config['user']}:{config['password']}@{config['host']}:{config['port']}/{config['database']}"   
    # Create engine
    engine = create_engine(connection)
    # Query to obtain data from FactVentas table
    query = """
    SELECT
        id_venta,
        id_cliente,
        id_producto,
        id_tiempo,
        cantidad,
        monto
    FROM dw.FactVentas
    ORDER BY id_venta;
    """
    
    # obtain data with query and save in DataFrame
    df = pd.read_sql(query,engine)
    
    # Export to CSV
    df.to_csv('FactVentas_export.csv',index=False, encoding='utf-8')
    
    # Close connection
    engine.dispose()
    
except Exception as e:
    print('Error')



#### Creating new schema dw_cloud

In [2]:
# Libraries
import psycopg2
from sqlalchemy import create_engine, text

# conection config
config = {
    'host': 'localhost',
    'port':'5432',
    'database':'data_warehouse',
    'user': 'postgres',
    'password': 'root'
}

try:
        # Create connection
        connection_string = f"postgresql://{config['user']}:{config['password']}@{config['host']}:{config['port']}/{config['database']}"
        
        # Create engine
        engine = create_engine(connection_string)

        with engine.connect() as conn:
            # Create Schema dw_cloud
            conn.execute(text("CREATE SCHEMA IF NOT EXISTS dw_cloud;"))

            # Create table FactVentas in dw_cloud
            create_table_sql = """
            CREATE TABLE IF NOT EXISTS dw_cloud.FactVentas (
                id_venta INT PRIMARY KEY,
                id_cliente INT NOT NULL,
                id_producto INT NOT NULL,
                id_tiempo INT NOT NULL,
                cantidad INT,
                monto NUMERIC(12,2)
            );
            """
            conn.execute(text(create_table_sql))
            
            # Create indexes
            indexes_sql = [
                "CREATE INDEX IF NOT EXISTS idx_factventas_cloud_cliente ON dw_cloud.FactVentas(id_cliente);",
                "CREATE INDEX IF NOT EXISTS idx_factventas_cloud_producto ON dw_cloud.FactVentas(id_producto);",
                "CREATE INDEX IF NOT EXISTS idx_factventas_cloud_tiempo ON dw_cloud.FactVentas(id_tiempo);"
            ]
            for index_sql in indexes_sql:
                conn.execute(text(index_sql))
            
            # Commit changes
            conn.commit()
        
        # Close connection
        engine.dispose()
        
except Exception as e:
    print("Error")
            
            
            

#### load `FactVentas_export.csv` file there in dw_cloud

In [ ]:
df_csv = pd.read_csv('FactVentas_export.csv')

# conection config
config = {
    'host': 'localhost',
    'port':'5432',
    'database':'data_warehouse',
    'user': 'postgres',
    'password': 'root'
}
        

try:
    # Create connection
    connection_string = f"postgresql://{config['user']}:{config['password']}@{config['host']}:{config['port']}/{config['database']}"
    # Create engine
    engine = create_engine(connection_string)
    
    with engine.connect() as conn:
        # Clean table at first
        conn.execute(text("TRUNCATE TABLE dw_cloud.FactVentas;"))
        # Load data using pandas
        df_csv.to_sql(
            name='factventas',
            con=engine,
            schema='dw_cloud',
            if_exists='append',
            index=False,
            method='multi'
        )
        # Commit changes
        conn.commit()
        result = conn.execute(text("SELECT COUNT(*) FROM dw_cloud.FactVentas;"))
        count = result.scalar()
        print(f"   Registros cargados en dw_cloud.FactVentas: {count}")
    engine.dispose()
except Exception as e:
    print('Error')
        

#### Querys to the table in dw_cloud schema

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text

# Tu configuración (usando la que ya tienes)
config = {
    'host': 'localhost',  # Cambié de 'Localhost' a 'localhost'
    'port':'5432',
    'database':'data_warehouse',
    'user': 'postgres',
    'password': 'root'
}

def execute_queries():
    """Ejecuta las consultas requeridas sobre dw_cloud.FactVentas"""
    
    try:
        # Crear conexión
        connection_string = f"postgresql://{config['user']}:{config['password']}@{config['host']}:{config['port']}/{config['database']}"
        engine = create_engine(connection_string)
        
        print("🔗 Conectando a dw_cloud para ejecutar consultas...")
        
        with engine.connect() as conn:
            
            # ===========================================
            # CONSULTA 1: Top 3 clientes con más ventas
            # ===========================================
            print("\n" + "="*60)
            print("📊 CONSULTA 1: TOP 3 CLIENTES CON MÁS VENTAS")
            print("="*60)
            
            query_top_clients = """
            SELECT 
                id_cliente,
                COUNT(*) as total_ventas,
                SUM(cantidad) as cantidad_total,
                SUM(monto) as monto_total,
                AVG(monto) as monto_promedio
            FROM dw_cloud.FactVentas
            GROUP BY id_cliente
            ORDER BY monto_total DESC
            LIMIT 3;
            """
            
            result = conn.execute(text(query_top_clients))
            top_clients = result.fetchall()
            
            print("🏆 Top 3 Clientes por Monto Total de Ventas:")
            print("-" * 80)
            print(f"{'Pos':<4} {'Cliente':<10} {'Ventas':<8} {'Cantidad':<10} {'Monto Total':<15} {'Promedio':<12}")
            print("-" * 80)
            
            for i, row in enumerate(top_clients, 1):
                print(f"{i:<4} {row[0]:<10} {row[1]:<8} {row[2]:<10} ${row[3]:<14,.2f} ${row[4]:<11,.2f}")
            
            # También mostrar usando pandas para mejor visualización
            df_top_clients = pd.read_sql_query(query_top_clients, engine)
            print(f"\n📋 Resultado en DataFrame:")
            print(df_top_clients)
            
            # ===========================================
            # CONSULTA 2: Ventas totales por mes
            # ===========================================
            print("\n" + "="*60)
            print("📅 CONSULTA 2: VENTAS TOTALES POR MES")
            print("="*60)
            
            # Para obtener el mes, necesitamos hacer JOIN con DimTiempo o extraer del id_tiempo
            # Como no especificaste el formato del id_tiempo, asumiré que es YYYYMMDD
            query_sales_by_month = """
            SELECT 
                EXTRACT(YEAR FROM TO_DATE(id_tiempo::text, 'YYYYMMDD')) as anio,
                EXTRACT(MONTH FROM TO_DATE(id_tiempo::text, 'YYYYMMDD')) as mes,
                COUNT(*) as total_ventas,
                SUM(cantidad) as cantidad_total,
                SUM(monto) as monto_total,
                AVG(monto) as monto_promedio
            FROM dw_cloud.FactVentas
            GROUP BY 
                EXTRACT(YEAR FROM TO_DATE(id_tiempo::text, 'YYYYMMDD')),
                EXTRACT(MONTH FROM TO_DATE(id_tiempo::text, 'YYYYMMDD'))
            ORDER BY anio, mes;
            """
            
            # Si el formato de id_tiempo es diferente, usaremos una consulta alternativa
            try:
                result = conn.execute(text(query_sales_by_month))
                monthly_sales = result.fetchall()
                
                print("📈 Ventas Totales por Mes:")
                print("-" * 85)
                print(f"{'Año':<6} {'Mes':<4} {'Ventas':<8} {'Cantidad':<10} {'Monto Total':<15} {'Promedio':<12}")
                print("-" * 85)
                
                for row in monthly_sales:
                    mes_nombre = [
                        "", "Ene", "Feb", "Mar", "Abr", "May", "Jun",
                        "Jul", "Ago", "Sep", "Oct", "Nov", "Dic"
                    ][int(row[1])]
                    print(f"{int(row[0]):<6} {mes_nombre:<4} {row[2]:<8} {row[3]:<10} ${row[4]:<14,.2f} ${row[5]:<11,.2f}")
                
                # DataFrame para mejor visualización
                df_monthly = pd.read_sql_query(query_sales_by_month, engine)
                df_monthly['mes_nombre'] = df_monthly['mes'].apply(
                    lambda x: ["", "Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio",
                              "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"][int(x)]
                )
                
                print(f"\n📋 Resultado en DataFrame:")
                print(df_monthly[['anio', 'mes_nombre', 'total_ventas', 'cantidad_total', 'monto_total']])
                
            except Exception as date_error:
                print("⚠️  Error procesando fechas. Usando consulta alternativa...")
                
                # Consulta alternativa agrupando por id_tiempo directamente
                query_alternative = """
                SELECT 
                    id_tiempo,
                    COUNT(*) as total_ventas,
                    SUM(cantidad) as cantidad_total,
                    SUM(monto) as monto_total
                FROM dw_cloud.FactVentas
                GROUP BY id_tiempo
                ORDER BY id_tiempo;
                """
                
                df_alt = pd.read_sql_query(query_alternative, engine)
                print("📊 Ventas agrupadas por id_tiempo:")
                print(df_alt)
            
            # ===========================================
            # CONSULTAS ADICIONALES (BONUS)
            # ===========================================
            print("\n" + "="*60)
            print("🎁 CONSULTAS ADICIONALES (BONUS)")
            print("="*60)
            
            # Resumen general
            query_summary = """
            SELECT 
                COUNT(*) as total_registros,
                COUNT(DISTINCT id_cliente) as clientes_unicos,
                COUNT(DISTINCT id_producto) as productos_unicos,
                COUNT(DISTINCT id_tiempo) as fechas_unicas,
                SUM(monto) as monto_total_general,
                AVG(monto) as monto_promedio_general,
                MIN(monto) as monto_minimo,
                MAX(monto) as monto_maximo
            FROM dw_cloud.FactVentas;
            """
            
            result = conn.execute(text(query_summary))
            summary = result.fetchone()
            
            print("📊 Resumen General del Data Warehouse:")
            print(f"   • Total de registros: {summary[0]:,}")
            print(f"   • Clientes únicos: {summary[1]:,}")
            print(f"   • Productos únicos: {summary[2]:,}")
            print(f"   • Fechas únicas: {summary[3]:,}")
            print(f"   • Monto total: ${summary[4]:,.2f}")
            print(f"   • Monto promedio: ${summary[5]:,.2f}")
            print(f"   • Monto mínimo: ${summary[6]:,.2f}")
            print(f"   • Monto máximo: ${summary[7]:,.2f}")
            
        engine.dispose()
        print(f"\n🎉 ¡Consultas completadas exitosamente!")
        
    except Exception as e:
        print(f"❌ Error ejecutando consultas: {str(e)}")
        return False

if __name__ == "__main__":
    execute_queries()